In [2]:
import wikipediaapi
import nltk
import pickle 
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

# nltk.download('punkt') # at first run
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

wiki_wiki = wikipediaapi.Wikipedia(
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

Using TensorFlow backend.


In [1]:
import tensorflow 
tensorflow.__version__

/home/livia/citation-needed-paper/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/livia/citation-needed-paper/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/livia/citation-needed-paper/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/livia/citation-needed-paper/

'1.7.0'

In [3]:
p_wiki = wiki_wiki.page("RM (rapper)")

In [3]:
organized_text = []

for s in p_wiki.sections:
    organized_text.append((s.title, sent_detector.tokenize(s.text.strip())))

In [4]:
organized_text

[('Early life',
  ['Kim Nam-joon was born on September 12, 1994, in Dongjak-gu, South Korea, and grew up in Ilsan-gu, where his family moved when he was five.',
   'He is the elder of two siblings and has a younger sister.As a child, RM largely learned English through watching Friends with his mother.',
   'As a student, he actively wrote poetry and often received awards for his writings.',
   'He posted his work to an online poetry website for roughly one year, where he received moderate attention.',
   'Through this, RM grew interested in pursuing a literary career but decided against it.',
   'At the age of 11, RM became interested in hip-hop music after hearing Epik High\'s "Fly" in fifth grade.',
   'He found that the song had provided him comfort, and, due to this, decided to look further into the genre.',
   'Following an introduction to American rapper Eminem by his school teacher, RM grew interested in lyricism, printing lyrics he felt were "cool" and sharing them with his fri

In [3]:
def construct_instance_reasons(statement, section_dict_path, vocab_w2v_path, max_len=-1):
    # Load the vocabulary
    vocab_w2v = pickle.load(open(vocab_w2v_path, 'rb'),encoding='latin1')

    # load the section dictionary.
    section_dict = pickle.load(open(section_dict_path, 'rb'))

    # construct the training data
    X = []
    sections = []
    y = []
    outstring=[]
    
    X_inst = []

    for word in statement:
        if max_len != -1 and len(X_inst) >= max_len:
            continue
        if word not in vocab_w2v:
            X_inst.append(vocab_w2v['UNK'])
        else:
            X_inst.append(vocab_w2v[word])

    # extract the section, and in case the section does not exist in the model, then assign UNK
    section = statement.strip().lower()
    sections.append(np.array([section_dict[section] if section in section_dict else 0]))

    X.append(X_inst)
    outstring.append(statement)
    #entity_id  revision_id timestamp   entity_title    section_id  section prg_idx sentence_idx    statement   citations

    X = pad_sequences(X, maxlen=max_len, value=vocab_w2v['UNK'], padding='pre')

    return X, np.array(sections), outstring

In [5]:
path_model = "models/fa_en_model_rnn_attention_section.h5"
model = load_model(path_model)

# load the data
max_seq_length = model.input[0].shape[1].value

X, sections, outstring =construct_instance_reasons("Students receiving high marks in their Honours program have the option of continuing to candidature of a Doctoral program, such as Doctor of Philosophy, without having to complete a master's degree.", "embeddings/word_dict_en.pck", "embeddings/section_dict_en.pck", max_seq_length)

In [6]:
pred = model.predict([X, sections])

In [7]:
pred

array([[9.9985206e-01, 1.4628991e-04]], dtype=float32)